In [1]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import json
import os


#set the API URL to use
#url = "http://www.cookcountyassessor.com/Search/Property-Search.aspx"
url_a = 'https://datacatalog.cookcountyil.gov/resource/uzyt-m557.json?tax_year=2024&$offset='  #URL for assessed values
url_p = 'https://datacatalog.cookcountyil.gov/resource/x54s-btds.json?tax_year=2024&$offset='  #URL for property characteristics

key = os.environ['API_CC_KEY'] 
secret = os.environ['API_CC_SECRET']

#pd.options.display.max_colwidth = 400
pd.set_option('display.max_columns', None)



In [2]:

def api_to_df(url, key, secret):

    #Establish a connection to the API
    #this code paginates from the API to pull 1000 records at a time.

    #set the authentification variables
    basic = HTTPBasicAuth(key, secret)

    offset = 0

    #keep looping, we will break the loop at the bottom
    while offset == 0 or len(d) == 1000:

        #authenticate again to url, use the offset as needed
        res = requests.get(url + str(offset), auth=basic)

        #save the content to a variable
        d = res.json()

        #on the first iteration of the loop just set the total equal to d, otherwise append it
        if offset == 0:
            d_all = d
        else:
            d_all = d_all + d

        #reset offset to the length of the response
        offset = len(d_all) + 1

        #for testing
        print(len(d), len(d_all), offset)

        #break the loop here if we have an offset less than 1000
        if len(d_all) < 1000:
            print('here')
            break


    return pd.json_normalize(d_all)


In [3]:
#This section is pulling the data, either directly from the API endpoint or through the CSV that has been cached. It takes about an hour to get each data set from the API
#(over 1M records for just one year of data) so that is why pulling from the csv is helpful. Note that dropping to a csv and re-uploading converts string values to numeric

#Pull from the API
#df_a = api_to_df(url_a,key,secret)
#df_p = api_to_df(url_p,key,secret)

#putting this data in a local csv saves time (takes about an hour to pull just one year of assessment data), but it converts several data types to numeric upon reingestion, including the pin number.

#df_a.to_csv(r"C:\Users\matth\OneDrive\Documents\AssessValues2024.csv", index=False)
#df_p.to_csv(r"C:\Users\matth\OneDrive\Documents\PropAtrb2024.csv", index=False)

#or, pull the same data from the csv which was cached from the same API call previously
df_a = pd.read_csv(r"C:\Users\matth\OneDrive\Documents\AssessValues2024.csv")
df_p = pd.read_csv(r"C:\Users\matth\OneDrive\Documents\PropAtrb2024.csv")


C:\Users\matth\AppData\Local\Temp\ipykernel_16324\112580501.py:15: DtypeWarning: Columns (3,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_p = pd.read_csv(r"C:\Users\matth\OneDrive\Documents\PropAtrb2024.csv")


In [4]:
#Merge the two data sets and augment the data with some additional calculated fields

df_all = df_a.merge(df_p, how='left', left_on='pin',right_on='pin')

#Add some fields that will help us
df_all['price_sqft_bldg'] = df_all['mailed_tot'] / df_all['char_bldg_sf']
df_all['price_sqft_land'] = df_all['mailed_tot'] / df_all['char_land_sf']

df_all[df_all['pin'] == 10104240110000]

,pin,tax_year,class_x,township_code_x,township_name,neighborhood_code,mailed_bldg,mailed_land,mailed_tot,certified_bldg,certified_land,certified_tot,year,card,class_y,township_code_y,tieback_proration_rate,card_proration_rate,cdu,pin_is_multicard,pin_num_cards,pin_is_multiland,pin_num_landlines,char_yrblt,char_bldg_sf,char_land_sf,char_beds,char_rooms,char_fbath,char_hbath,char_frpl,char_type_resd,char_cnst_qlty,char_apts,char_attic_fnsh,char_gar1_att,char_gar1_area,char_gar1_size,char_gar1_cnst,char_attic_type,char_bsmt,char_ext_wall,char_heat,char_repair_cnd,char_bsmt_fin,char_roof_cnst,char_use,char_site,char_ncu,char_renovation,recent_renovation,char_porch,char_air,char_tp_plan,tieback_key_pin,price_sqft_bldg,price_sqft_land
341783,10104240110000,2024,204,24,Niles,24051,39996,11004,51000,NaN,NaN,NaN,2024.0,1.0,204,24.0,1.0,0.0,AV,False,1.0,False,1.0,1955.0,2617.0,9170.0,5.0,9.0,2.0,0.0,0.0,1 Story,Average,NaN,NaN,Yes,No,2 cars,Frame,NaN,Full,Masonry,Warm Air Furnace,Average,Unfinished,Shingle + Asphalt,Single-Family,Not Relevant To Value,0.0,No,False,NaN,No Central A/C,NaN,NaN,19.487963,5.561614


In [21]:
#Find houses with a similar structure to ours but with a lower assessed value

df_apl = df_all[df_all['township_code_x'] == 24]
df_apl = df_apl[df_apl['neighborhood_code'] == 24051]
df_apl = df_apl[df_apl['char_beds'] >= 4]
df_apl = df_apl[df_apl['char_fbath'] >= 2]
df_apl = df_apl[df_apl['char_bldg_sf'] >= 2500]
df_apl = df_apl[df_apl['class_x'] == '204']

#df_all[(df_all['pin'] == 10154130350000) | (df_all['pin'] == 10104090210000) | (df_all['pin'] == 10153190350000) | (df_all['pin'] == 10154130350000)]


df_apl.sort_values('price_sqft_bldg',ascending=True).head(35)

#10154130350000 - 1
#10104090210000 - 2
#10153190350000 - 3

#10-10-424-003-0000 - on Keeler, similar lot size but only $10k less

#10144160270000 - EVANSTON


,pin,tax_year,class_x,township_code_x,township_name,neighborhood_code,mailed_bldg,mailed_land,mailed_tot,certified_bldg,certified_land,certified_tot,year,card,class_y,township_code_y,tieback_proration_rate,card_proration_rate,cdu,pin_is_multicard,pin_num_cards,pin_is_multiland,pin_num_landlines,char_yrblt,char_bldg_sf,char_land_sf,char_beds,char_rooms,char_fbath,char_hbath,char_frpl,char_type_resd,char_cnst_qlty,char_apts,char_attic_fnsh,char_gar1_att,char_gar1_area,char_gar1_size,char_gar1_cnst,char_attic_type,char_bsmt,char_ext_wall,char_heat,char_repair_cnd,char_bsmt_fin,char_roof_cnst,char_use,char_site,char_ncu,char_renovation,recent_renovation,char_porch,char_air,char_tp_plan,tieback_key_pin,price_sqft_bldg,price_sqft_land
320272,10144160270000,2024,204,24,Niles,24051,8127,6000,14127,NaN,NaN,NaN,2024.0,1.0,204,24.0,0.20,0.20,AV,False,1.0,False,1.0,1954.0,3034.0,5310.0,4.0,9.0,2.0,1.0,2.0,1 Story,Average,NaN,NaN,Yes,No,2 cars,Masonry,NaN,Partial,Masonry,Warm Air Furnace,Average,Unfinished,Shingle + Asphalt,Single-Family,Not Relevant To Value,0.0,No,False,NaN,Central A/C,Stock Plan,1.014416e+13,4.656229,2.660452
370365,10154130350000,2024,204,24,Niles,24051,8556,4674,13230,NaN,NaN,NaN,2024.0,1.0,204,24.0,0.20,0.20,AV,False,1.0,False,1.0,1951.0,2811.0,3895.0,5.0,9.0,3.0,0.0,1.0,1 Story,Average,NaN,NaN,Yes,No,2.5 cars,Masonry,Partial,Partial,Masonry,Hot Water Steam,Average,Unfinished,Shingle + Asphalt,Single-Family,Not Relevant To Value,0.0,No,False,NaN,Central A/C,NaN,1.015413e+13,4.706510,3.396662
348887,10154130380000,2024,204,24,Niles,24051,8556,5998,14554,NaN,NaN,NaN,2024.0,1.0,204,24.0,0.20,0.20,AV,False,1.0,False,1.0,1951.0,2811.0,5389.0,5.0,9.0,3.0,0.0,1.0,1 Story,Average,NaN,NaN,Yes,No,2.5 cars,Masonry,Partial,Partial,Masonry,Hot Water Steam,Average,Unfinished,Shingle + Asphalt,Single-Family,Not Relevant To Value,0.0,No,False,NaN,Central A/C,NaN,1.015413e+13,5.177517,2.700687
323030,10153050300000,2024,204,24,Niles,24051,9919,5430,15349,NaN,NaN,NaN,2024.0,1.0,204,24.0,0.10,0.25,AV,False,1.0,False,1.0,1954.0,2521.0,4525.0,4.0,8.0,2.0,1.0,2.0,1 Story,Average,NaN,NaN,Yes,No,2 cars,Masonry,NaN,Full,Masonry,Warm Air Furnace,Average,Formal Rec Room,Shingle + Asphalt,Single-Family,Not Relevant To Value,0.0,No,False,NaN,Central A/C,Stock Plan,1.015305e+13,6.088457,3.392044
393455,10144160280000,2024,204,24,Niles,24051,12191,6372,18563,NaN,NaN,NaN,2024.0,1.0,204,24.0,0.30,0.30,AV,False,1.0,False,1.0,1954.0,3034.0,5310.0,4.0,9.0,2.0,1.0,2.0,1 Story,Average,NaN,NaN,Yes,No,2 cars,Masonry,NaN,Partial,Masonry,Warm Air Furnace,Average,Unfinished,Shingle + Asphalt,Single-Family,Not Relevant To Value,0.0,No,False,NaN,Central A/C,Stock Plan,1.014416e+13,6.118326,3.495857
389671,10154130360000,2024,204,24,Niles,24051,12834,4673,17507,NaN,NaN,NaN,2024.0,1.0,204,24.0,0.30,0.30,AV,False,1.0,False,1.0,1951.0,2811.0,3894.0,5.0,9.0,3.0,0.0,1.0,1 Story,Average,NaN,NaN,Yes,No,2.5 cars,Masonry,Partial,Partial,Masonry,Hot Water Steam,Average,Unfinished,Shingle + Asphalt,Single-Family,Not Relevant To Value,0.0,No,False,NaN,Central A/C,NaN,1.015413e+13,6.228033,4.495891
370366,10154130370000,2024,204,24,Niles,24051,12834,4673,17507,NaN,NaN,NaN,2024.0,1.0,204,24.0,0.30,0.30,AV,False,1.0,False,1.0,1951.0,2811.0,3894.0,5.0,9.0,3.0,0.0,1.0,1 Story,Average,NaN,NaN,Yes,No,2.5 cars,Masonry,Partial,Partial,Masonry,Hot Water Steam,Average,Unfinished,Shingle + Asphalt,Single-Family,Not Relevant To Value,0.0,No,False,NaN,Central A/C,NaN,1.015413e+13,6.228033,4.495891
353498,10154050130000,2024,204,24,Niles,24051,13417,4722,18139,NaN,NaN,NaN,2024.0,1.0,204,24.0,0.30,0.30,AV,False,1.0,False,1.0,1960.0,2603.0,3935.0,5.0,9.0,2.0,1.0,2.0,1 Story,Average,NaN,NaN,Yes,No,2 cars,Masonry,NaN,Partial,Masonry,Warm Air Furnace,Average,Unfinished,Shingle + Asphalt,Single-Family,Not Relevant To Value,0.0,No,False,NaN,Central A/C,NaN,1.015405e+13,6.968498,4.609657
348886,10154050110000,2024,204,24,Niles,24051,13417,5982,19399,NaN,NaN,NaN,2024.0,1.0,204,24.0,0.30,0.30,AV,Fa